In [ ]:
import pytest
import os
import json
from unittest.mock import patch, MagicMock
from rss_collector.orchestration import process_feeds


@pytest.fixture
def mock_feed_data():
    """Mock feed data for testing."""
    return {
        "entries": [
            {
                "title": "Article 1",
                "link": "http://example.com/article1",
                "published": "2024-11-30T12:00:00Z",
            },
            {
                "title": "Article 2",
                "link": "http://example.com/article2",
                "published": "2024-11-30T13:00:00Z",
            },
        ]
    }


@pytest.fixture
def mock_proxy_settings():
    """Mock proxy settings for testing."""
    return {
        "app_proxy": "http://mock_proxy.com:85",
        "username": "mock_user",
        "password": "mock_password",
    }


@pytest.fixture
def test_feed_urls():
    """Mock feed URLs for testing."""
    return [
        "http://mock_feed_1.com/rss",
        "http://mock_feed_2.com/rss",
    ]


@patch("rss_collector.utils.get_proxy_settings")
@patch("rss_collector.proxy.fetch_feed_content_with_proxy")
@patch("rss_collector.feed_parser.parse_feed")
def test_process_feeds(
    mock_parse_feed, mock_fetch_feed, mock_get_proxy_settings, mock_feed_data, mock_proxy_settings, test_feed_urls
):
    """
    Test the process_feeds function.
    """
    # Mock proxy settings
    mock_get_proxy_settings.return_value = mock_proxy_settings

    # Mock fetch_feed_content_with_proxy to return feed content
    mock_fetch_feed.return_value = "Mock RSS Feed Content"

    # Mock parse_feed to return the mocked feed data
    mock_parse_feed.return_value = mock_feed_data

    # Create a temporary output directory
    output_dir = "rss_metadata"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    try:
        # Call the process_feeds function
        process_feeds(test_feed_urls)

        # Verify proxy settings and fetch_feed_content_with_proxy were called
        mock_get_proxy_settings.assert_called_once()
        assert mock_fetch_feed.call_count == len(test_feed_urls)

        # Verify parse_feed was called for each feed
        assert mock_parse_feed.call_count == len(test_feed_urls)

        # Verify the JSON file is saved correctly
        saved_files = os.listdir(output_dir)
        assert len(saved_files) == 1  # Only one JSON file should be created
        output_file = os.path.join(output_dir, saved_files[0])

        with open(output_file, "r") as f:
            metadata = json.load(f)

        # Validate metadata structure
        assert "timestamp" in metadata
        assert "articles" in metadata
        assert len(metadata["articles"]) == len(mock_feed_data["entries"])
    finally:
        # Cleanup
        for file in os.listdir(output_dir):
            os.remove(os.path.join(output_dir, file))
        os.rmdir(output_dir)
